In [313]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

import numpy as np

import torch.autograd as autograd
from tqdm.notebook import trange

In [314]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [315]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)


In [316]:
print(hidden)

(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


## POS Tagging

In [317]:
training_data = [("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
                ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])]

In [318]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [319]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

In [320]:
ix_to_tag = {ix:tag for tag,ix in tag_to_ix.items()}
ix_to_word = {ix:word for word,ix in word_to_ix.items()}

In [321]:
ix_to_tag

{0: 'DET', 1: 'NN', 2: 'V'}

In [322]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [323]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_in = embeds.view(len(sentence), 1, -1)
        #print(lstm_in.shape)
        lstm_out, _ = self.lstm(lstm_in)
        #print(lstm_out.shape)
        
        to_linear = lstm_out.view(len(sentence), -1)
        #print(to_linear.shape)
        tag_space = self.hidden2tag(to_linear)
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [324]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    #print(tag_scores)

for epoch in range(50):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [325]:
loss_function(tag_scores, targets)

tensor(0.9058, grad_fn=<NllLossBackward>)

In [326]:
tag_scores.shape

torch.Size([4, 3])

In [327]:
targets.shape

torch.Size([4])

In [328]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    #print(tag_scores)
    
    preds = np.argmax(tag_scores.detach().numpy(),axis=1)
    
    print(inputs)


tensor([0, 1, 2, 3, 4])


In [329]:
# See what the scores are after training
with torch.no_grad():
    for sentence, tags in training_data:
        inputs = prepare_sequence(sentence, word_to_ix)
        tag_scores = model(inputs)

        preds_ix = np.argmax(tag_scores.detach().numpy(),axis=1)
        preds = [ix_to_tag[p] for p in preds_ix]
        print(sentence)
        
        print(f'Preds: {preds}')
        print(f'Tags: {tags}')
        
        print(f'-------------------------------')

['The', 'dog', 'ate', 'the', 'apple']
Preds: ['NN', 'NN', 'NN', 'NN', 'NN']
Tags: ['DET', 'NN', 'V', 'DET', 'NN']
-------------------------------
['Everybody', 'read', 'that', 'book']
Preds: ['NN', 'NN', 'DET', 'NN']
Tags: ['NN', 'V', 'DET', 'NN']
-------------------------------


## Character-level LSTM

In [330]:
## this function add padding (spaces) to make sure all words are length 10 characters
def sent_to_char(sent):
    char_list = []
    for w in sent:
        a = []
        for c in w:
            a.append(c)
        #for p in range(10 - len(w)):
        #    char_list.extend(' ')
        char_list.append(a)
    return char_list

In [331]:
char_to_ix = {}
for sent, tags in training_data:
    for word in sent_to_char(sent):
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)

In [332]:
sent_to_char(['a','sentence'])

[['a'], ['s', 'e', 'n', 't', 'e', 'n', 'c', 'e']]

In [375]:
class char_lstm(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, vocab_size_char, tagset_size):
        super(char_lstm, self).__init__()
        self.hidden_dim = hidden_dim

        self.char_embeddings = nn.Embedding(vocab_size_char, embedding_dim)
        self.lstm_char = nn.LSTM(embedding_dim, hidden_dim)
        
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim + hidden_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
                
        self.hidden = self.init_hidden(self.hidden_dim)
        self.hidden_char = self.init_hidden(self.hidden_dim)
        

    def forward(self, sentence_in, char_in):
        num_words = sentence_in.shape[0]
                
        #char_emb = self.char_embeddings(char_in)
        embeds = self.word_embeddings(sentence_in)
                
        char_output_list = []
        for word in char_in:
            self.hidden_char = self.init_hidden(self.hidden_dim)
            char_emb = self.char_embeddings(word)
            lstm_out_1, self.hidden_char = self.lstm_char(char_emb.view(char_emb.shape[0], 1, -1),self.hidden_char)        
            char_output_list.append(lstm_out_1[-1])

            #char_output = torch.cat([char_output, lstm_out_1[-1]], dim=0).clone()

        ## this keeps only the last output from each charcter sequence
        #print(char_output_list)

        char_output_tensor = torch.cat(char_output_list)

        merged = torch.cat([embeds, char_output_tensor], dim=1) 
        
        self.hidden = self.init_hidden(self.hidden_dim)
        
        lstm_out, self.hidden = self.lstm(merged.view(num_words, 1, -1),self.hidden)
        lstm_out_2 = lstm_out.view(num_words,-1)
        
        tag_space = self.hidden2tag(lstm_out_2)
        #print(tag_space.shape)
        
        tag_scores = F.log_softmax(tag_space, dim=1)
        
        return tag_scores
    
    def init_hidden_old(self,size):
        return (autograd.Variable(torch.zeros(1, 1, size)),autograd.Variable(torch.zeros(1, 1, size)))
    
    def init_hidden(self,size):
        return (torch.zeros(1, 1, size),torch.zeros(1, 1, size))

In [376]:
EMBEDDING_DIM,HIDDEN_DIM = 6,3
model = char_lstm(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [377]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def prepare_sequence_char(seq, to_ix):
    sentence_tensor = []
    for word in seq:
        idxs = [to_ix[w] for w in word]
        word_tensor = torch.tensor(idxs, dtype=torch.long)
        sentence_tensor.append(word_tensor)
    return sentence_tensor


In [378]:
for sentence, tags in training_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
    targets = prepare_sequence(tags, tag_to_ix)
    tag_scores = model(sentence_in,char_in)
    #break

In [379]:
EMBEDDING_DIM,HIDDEN_DIM = 6,3
model = char_lstm(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


model.zero_grad()

num_words = sentence_in.shape[0]

embeds = model.word_embeddings(sentence_in)

#char_output = torch.zeros(0)
char_output_list = []
for word in char_in:
    char_emb = model.char_embeddings(word)
    lstm_out_1, model.hidden_char = model.lstm_char(char_emb.view(char_emb.shape[0], 1, -1),model.hidden_char)        
    char_output_list.append(lstm_out_1[-1])
    
    #char_output = torch.cat([char_output, lstm_out_1[-1]], dim=0).clone()
    
## this keeps only the last output from each charcter sequence
print(char_output_list)

char_output_tensor = torch.cat(char_output_list)

merged = torch.cat([embeds, char_output_tensor], dim=1) 

lstm_out, model.hidden = model.lstm(merged.view(num_words, 1, -1),model.hidden)
lstm_out_2 = lstm_out.view(num_words,-1)

tag_space = model.hidden2tag(lstm_out_2)

tag_scores = F.log_softmax(tag_space, dim=1)

loss = loss_function(tag_scores, targets)
loss.backward(retain_graph=True)


[tensor([[-0.0935, -0.1359, -0.1068]], grad_fn=<SelectBackward>), tensor([[-0.2999, -0.1568,  0.3108]], grad_fn=<SelectBackward>)]


In [380]:
training_data

[(['ate', 'read'], ['NN', 'V']), (['that', 'read'], ['NN', 'V'])]

In [383]:
training_data = [("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
                ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])]

In [384]:
EMBEDDING_DIM,HIDDEN_DIM = 6,4
model = char_lstm(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(char_to_ix), len(tag_to_ix))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in trange(100):
    model.zero_grad()
    retain = True
    #model.
    for sentence, tags in training_data:
        sentence_in = prepare_sequence(sentence, word_to_ix)
        char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        tag_scores = model(sentence_in,char_in)

        loss = loss_function(tag_scores, targets)
        
        loss.backward()
        #loss.backward()
        optimizer.step()

In [385]:
# See what the scores are after training
with torch.no_grad():
    for sentence, tags in training_data:
        sentence_in = prepare_sequence(sentence, word_to_ix)
        char_in = prepare_sequence_char(sent_to_char(sentence), char_to_ix)
        tag_scores = model(sentence_in,char_in)

        preds_ix = np.argmax(tag_scores.detach().numpy(),axis=1)
        preds = [ix_to_tag[p] for p in preds_ix]
        print(sentence)
        
        print(f'Preds: {preds}')
        print(f'Tags:  {tags}')
        
        print(f'-------------------------------')

['The', 'dog', 'ate', 'the', 'apple']
Preds: ['DET', 'NN', 'V', 'DET', 'NN']
Tags:  ['DET', 'NN', 'V', 'DET', 'NN']
-------------------------------
['Everybody', 'read', 'that', 'book']
Preds: ['NN', 'V', 'DET', 'NN']
Tags:  ['NN', 'V', 'DET', 'NN']
-------------------------------


In [386]:
tag_scores

tensor([[-1.6102, -0.2720, -3.2618],
        [-1.4204, -2.0099, -0.4710],
        [-0.7085, -0.8257, -2.6636],
        [-1.3030, -0.3682, -3.3160]])